In [1]:
import requests
import json
import pandas as pd

In [2]:
tomtom_api_key = '95GWSfDPOA0Nr9v0To26HEsAucy2yiR7'

In [3]:
# Traffic incidents url
baseURL = 'api.tomtom.com'
versionNumber = 5
bbox = '4.8854592519716675,52.36934334773164,4.897883244144765,52.37496348620152'
# Fields to request
fields='{incidents{type,geometry{type,coordinates},properties{iconCategory,events{description,code,iconCategory}}}}'
language = 'en-GB'


incident_url = f'https://{baseURL}/traffic/services/{versionNumber}/incidentDetails?key={tomtom_api_key}&bbox={bbox}&fields={fields}&language={language}'

In [4]:
# Traffic flow url (updates per minute)
baseURL = 'api.tomtom.com'
versionNumber = 4
style = 'relative0'
zoom = 10
format = 'json'
point = '59.34881%2C18.07302'
unit = 'KMPH'
boolean = 'false'

traffic_flow_url = f'https://{baseURL}/traffic/services/{versionNumber}/flowSegmentData/{style}/{zoom}/{format}?key={tomtom_api_key}&point={point}&unit={unit}&openLr={boolean}'

In [2]:
# Weather url

longitude = 18.07302
latitude = 59.34881
weather_url = f'https://opendata-download-metanalys.smhi.se/api/category/mesan1g/version/2/geotype/point/lon/{longitude}/lat/{latitude}/data.json'

In [3]:
# Send HTTP Get request
response = requests.get(weather_url)

# Check the status code of the request
if response.status_code == 200:
    data = response.json()
    pretty_data = json.dumps(data, indent=2)
    # print(data['flowSegmentData'])
    print(pretty_data)
   
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{
  "approvedTime": "2024-01-01T15:37:16Z",
  "referenceTime": "2024-01-01T15:00:00Z",
  "geometry": {
    "type": "Point",
    "coordinates": [
      [
        18.052046,
        59.346752
      ]
    ]
  },
  "timeSeries": [
    {
      "validTime": "2024-01-01T15:00:00Z",
      "parameters": [
        {
          "name": "t",
          "levelType": "hl",
          "level": 2,
          "unit": "Cel",
          "values": [
            -2.7
          ]
        },
        {
          "name": "Tiw",
          "levelType": "hl",
          "level": 2,
          "unit": "Cel",
          "values": [
            -3.6
          ]
        },
        {
          "name": "gust",
          "levelType": "hl",
          "level": 10,
          "unit": "m/s",
          "values": [
            9.5
          ]
        },
        {
          "name": "wd",
          "levelType": "hl",
          "level": 10,
          "unit": "degree",
          "values": [
            83
          ]
        },
        {


In [44]:
data_dict = data['flowSegmentData']
new_data_dict = data_dict.fromkey
index = [0]
# del data_dict['coordinates']
data_df = pd.DataFrame(data_dict, index=index)
new_key_names = {'currentSpeed': 'current_speed', 
                    'freeFlowSpeed': 'free_flow_speed'}
data_df = data_df.rename(columns=new_key_names)
# print(data_df.keys())
data_df

,frc,current_speed,free_flow_speed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,@version
0,FRC1,27,36,395,296,0.98059,False,traffic-service-flow 1.0.104


In [21]:
# Decode the unicode ("description" in events)
escaped_string = r'\u4ea4\u901a\u505c\u6ede'
decoded_string = bytes(escaped_string, 'utf-8').decode('unicode-escape')
print(decoded_string)

交通停滞


In [72]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/213141
Connected. Call `.close()` to terminate connection gracefully.


In [73]:
# Retrieve traffic flow data
# traffic_fg = fs.get_feature_group(name='test', version=3)
# query = traffic_fg.select_all()
feature_view = fs.get_feature_view(
    name='test_combined',
    version=2,
)

In [74]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

Error: Reading data from Hopsworks, using Hive           


DatabaseError: Execution failed on sql: SELECT `fg1`.`current_speed` `current_speed`, `fg1`.`free_flow_speed` `free_flow_speed`, `fg1`.`confidence` `confidence`, `fg1`.`weekend` `weekend`, `fg1`.`day` `day`, `fg1`.`hour` `hour`, `fg1`.`minute` `minute`, `fg0`.`temp` `temp`, `fg0`.`wd` `wd`, `fg0`.`ws` `ws`, `fg0`.`prec1h` `prec1h`, `fg0`.`frsn1h` `frsn1h`, `fg0`.`vis` `vis`
FROM `han760_featurestore`.`test_3` `fg1`
FULL JOIN `han760_featurestore`.`test_weather_2` `fg0` ON `fg1`.`day` = `fg0`.`day` AND `fg1`.`hour` = `fg0`.`hour`
TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=['*org.apache.hive.service.cli.HiveSQLException:Error while processing statement: FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.tez.TezTask:28:27', 'org.apache.hive.service.cli.operation.Operation:toSQLException:Operation.java:343', 'org.apache.hive.service.cli.operation.SQLOperation:runQuery:SQLOperation.java:232', 'org.apache.hive.service.cli.operation.SQLOperation:ru

In [66]:
X_train

,free_flow_speed,confidence,weekend,day,hour,minute
0,35,0.993480,False,26,15,5
1,36,1.000000,False,26,14,6
2,36,1.000000,False,26,14,7
3,36,1.000000,False,26,14,8
4,35,1.000000,False,26,14,9
...,...,...,...,...,...,...
177,35,0.990000,False,26,17,4
179,36,0.996106,False,26,17,6
180,36,0.996106,False,26,17,7
181,36,0.996106,False,26,17,8


In [67]:
X_test

,free_flow_speed,confidence,weekend,day,hour,minute
22,35,0.990000,False,26,14,27
27,35,0.990000,False,26,14,32
28,36,0.990000,False,26,14,33
34,36,1.000000,False,26,14,39
43,36,0.994600,False,26,14,48
45,35,0.994600,False,26,14,50
48,35,0.993480,False,26,14,53
53,35,0.990000,False,26,14,58
62,35,0.990000,False,26,15,8
73,36,0.992213,False,26,15,20


In [68]:
y_train

,current_speed
0,24
1,26
2,26
3,26
4,25
...,...
177,26
179,25
180,25
181,26


In [56]:
import pandas as pd

# 创建两个示例 DataFrame
data1 = {
    'ID': [1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'Data1': ['A', 'ZZ', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T']
}

data2 = {
    'ID': [1, 1, 3, 5, 7, 9],
    'Data2': ['X', 'YY','Y', 'Z', 'U', 'V']
}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# 使用 merge 进行连接
result_df = pd.merge(df1, df2, on='ID', how='outer')

# 打印合并后的结果
print(result_df)


    ID Data1 Data2
0    1     A     X
1    1     A    YY
2    1    ZZ     X
3    1    ZZ    YY
4    2     B   NaN
5    3     C     Y
6    4     D   NaN
7    5     E     Z
8    6     F   NaN
9    7     G     U
10   8     H   NaN
11   9     I     V
12  10     J   NaN
13  11     K   NaN
14  12     L   NaN
15  13     M   NaN
16  14     N   NaN
17  15     O   NaN
18  16     P   NaN
19  17     Q   NaN
20  18     R   NaN
21  19     S   NaN
22  20     T   NaN
